# 1.Data Injestion and Validation

In [1]:
import pandas as pd 
import numpy as np
import json
import os
import re
import shutil

In [3]:
mkdir good_data_pred

In [4]:
mkdir Bad_data_pred

In [8]:
#Global Path
schema_file_path = 'C:\\Users\\User\\LearnBay\\ML Projects\\Wafer Fault Detection\\schema_prediction.json'
training_folder_path = 'C:\\Users\\User\\LearnBay\\ML Projects\\Wafer Fault Detection\\Prediction_Batch_files/'
good_data_folder_path = 'C:\\Users\\User\\LearnBay\\ML Projects\\Wafer Fault Detection\\good_data_pred/'
bad_data_folder_path = 'C:\\Users\\User\\LearnBay\\ML Projects\\Wafer Fault Detection\\Bad_data_pred/'

In [9]:
# extracting values from predicted schema

def valuesfromschema():
    with open(schema_file_path, 'r') as f:
        dic = json.load(f)
        pattern = dic['SampleFileName']
        LengthOfDateStampInFile = dic['LengthOfDateStampInFile']
        LengthOfTimeStampInFile = dic['LengthOfTimeStampInFile']
        column_names = dic['ColName']
        NumberofColumns = dic['NumberofColumns']
        
        return pattern, LengthOfDateStampInFile, LengthOfTimeStampInFile, column_names, NumberofColumns

In [10]:
pattern, LengthOfDateStampInFile, LengthOfTimeStampInFile, column_names, NumberofColumns = valuesfromschema()


In [11]:
#Regex
def regex():
    regex = "['wafer']+['\_'']+[\d_]+[\d]+\.csv"
    return regex

In [12]:
regex = regex()

In [13]:
# code for filtering the good and bad  files
def validate_file_name(regex, LengthOfDateStampInFile, LengthOfTimeStampInFile):
    for filename in os.listdir(training_folder_path):
        if (re.match(regex, filename)):
            split_at_dot = re.split('.csv', filename)
            split_at_dot = re.split('_', split_at_dot[0])
            if len(split_at_dot[1]) == LengthOfDateStampInFile:
                if len(split_at_dot[2]) == LengthOfTimeStampInFile:
                    shutil.copy(f'{training_folder_path}'+filename , good_data_folder_path)
                else:
                    shutil.copy(f'{training_folder_path}'+filename , bad_data_folder_path)
            else:
                shutil.copy(f'{training_folder_path}'+filename , bad_data_folder_path)
        else:
                shutil.copy(f'{training_folder_path}'+filename , bad_data_folder_path)
            
            
                


In [15]:
validate_file_name(regex, LengthOfDateStampInFile, LengthOfTimeStampInFile)

In [16]:
# validating column length in the file
def validatecolumn(NumberofColumns):
    for filename in os.listdir(good_data_folder_path):
        csv = pd.read_csv(f'{good_data_folder_path}'+filename)
        if csv.shape[1] == NumberofColumns:
            pass
        else:
            shutil.copy(f'{good_data_folder_path}'+filename , bad_data_folder_path)

In [17]:
validatecolumn(NumberofColumns)

In [18]:
def validate_missing_column():
    for filename in os.listdir(good_data_folder_path):
        csv = pd.read_csv(f'{good_data_folder_path}'+filename)
        count=0
        for column in csv:
            if (len(csv[column])- csv[column].count()) == len(csv[column]):
                count +=1
                shutil.copy(f'{good_data_folder_path}'+filename , bad_data_folder_path)
                break
        if count==0:
            csv.rename(columns = {'Unnamed: 0': "wafer"}, inplace = True)
            csv.to_csv(f'{good_data_folder_path}'+ filename, index = None , header=True)

In [19]:
validate_missing_column()

In [22]:
# Merging or Combining all Data
li = []
for filename in os.listdir(good_data_folder_path):
    csv = pd.read_csv(f'{good_data_folder_path}'+filename)
    li.append(csv)

combined_data = pd.concat(li , axis = 0, ignore_index=True)
combined_data.to_csv('pred_main_data.csv', index = False)

# 2.Data_Preprocessing

In [24]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer

df = pd.read_csv('pred_main_data.csv')

df

# deleting unwanted column
df.drop(columns = ['Unnamed: 0' , 'wafer'] , inplace = True)


imputer = KNNImputer(n_neighbors=3, weights='uniform' , missing_values=np.nan)
new_data = imputer.fit_transform(df)

new_data = pd.DataFrame(new_data , columns= df.columns)
new_data.to_csv('pred_clean_combined_data.csv')


# # checking columns whihc have standard deviation of zero
col_to_drop = []
describe = df.describe()
for x in new_data.columns:
    if describe[x]['std']==0:
        col_to_drop.append(x)

col_to_drop
new_data.drop(columns=col_to_drop, axis=1 , inplace=True)


new_data.to_csv('pred_clean_data.csv' , index=False)




# 3.Clustering

In [25]:
import pandas as pd
from sklearn.cluster import KMeans
from kneed import KneeLocator
import matplotlib.pyplot as plt
import joblib

data = pd.read_csv('pred_clean_data.csv')
model = joblib.load('models/kmeans.pkl')
y_pred = model.predict(data)
data['cluster'] = y_pred

In [27]:
data.to_csv('pred_clusters.csv', index=False)

# 4.Prediction

In [28]:
clusters = data['cluster'].unique()

#loading all the models

model_0 = joblib.load('models/random_forest_0.pkl')
model_1 = joblib.load('models/xgboost_1.pkl')
model_2 = joblib.load('models/random_forest_2.pkl')


for i in clusters:
    cluster_data = data[data['cluster']==i]
#     wafer_names = list(data[''])
    cluster_data = data.drop(columns='cluster' , axis = 1)
    if i == 0:
        predict_0 = model_0.predict(cluster_data)
    if i == 1:
        predict_1 = model_1.predict(cluster_data)
    if i == 2:
        predict_2 = model_2.predict(cluster_data)



In [29]:
np.unique(predict_0)

array([-1.])

In [31]:
np.unique(predict_1)

array([-1.,  1.])

In [30]:
np.unique(predict_2)

array([-1.])